Inspired from https://www.kaggle.com/kimtaegwan/what-s-your-cv-method

In [2]:
import numpy as np 
import pandas as pd 

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm_notebook
import os
import gc

import lightgbm as lgb
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

# Data

In [3]:
sample_submission = pd.read_csv("/kaggle/input/ashrae-great-energy-predictor-iii-dataset/sample_submission.csv")
df_train = pd.read_csv('/kaggle/input/ashrae-great-energy-predictor-iii-dataset/train.csv')
weather_train = pd.read_csv('/kaggle/input/ashrae-great-energy-predictor-iii-dataset/weather_train.csv')
building_metadata = pd.read_csv('/kaggle/input/ashrae-great-energy-predictor-iii-dataset/building_metadata.csv')

print('df_train_shape :' , df_train.shape)
print('weather_train_shape :' , weather_train.shape)
print('building_metadata_shape :' , building_metadata.shape)

df_train_shape : (20216100, 4)
weather_train_shape : (139773, 9)
building_metadata_shape : (1449, 6)


In [4]:
del df_train
df_train = pd.read_csv('/kaggle/input/ashrae-great-energy-predictor-iii-dataset/train.csv')
df_train.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading'], dtype='object')

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
building_metadata.primary_use = le.fit_transform(building_metadata.primary_use)



building_metadata['cnt_building_per_site'] = building_metadata.groupby(['site_id']).building_id.transform(lambda x: x.size)
building_metadata['cnt_building_per_site_prim'] = building_metadata.groupby(['site_id', 'primary_use']).building_id.transform(lambda x: x.size)
building_metadata['sqr_mean_per_site'] = building_metadata.groupby(['site_id', ]).square_feet.transform('median')
building_metadata['sqr_mean_per_prim_site'] = building_metadata.groupby(['site_id', 'primary_use']).square_feet.transform('median')

building_metadata.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count,cnt_building_per_site,cnt_building_per_site_prim,sqr_mean_per_site,sqr_mean_per_prim_site
0,0,0,0,7432,2008.0,NaN,105,30,52957.0,72175.5
1,0,1,0,2720,2004.0,NaN,105,30,52957.0,72175.5
2,0,2,0,5376,1991.0,NaN,105,30,52957.0,72175.5
3,0,3,0,23685,2002.0,NaN,105,30,52957.0,72175.5
4,0,4,0,116607,1975.0,NaN,105,30,52957.0,72175.5


In [6]:
df_train = df_train.merge(building_metadata, on='building_id', how='left')
df_train = df_train.merge(weather_train, on=['site_id', 'timestamp'], how='left')
del weather_train
gc.collect()

print('df_train_merge shape: ',df_train.shape)

df_train_merge shape:  (20216100, 20)


In [7]:
df_train.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'cnt_building_per_site', 'cnt_building_per_site_prim',
       'sqr_mean_per_site', 'sqr_mean_per_prim_site', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed'],
      dtype='object')

In [8]:
df_train.shape

(20216100, 20)

In [9]:
df_train.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'cnt_building_per_site', 'cnt_building_per_site_prim',
       'sqr_mean_per_site', 'sqr_mean_per_prim_site', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed'],
      dtype='object')

# feature engneering

In [10]:
#drop col
unimportant_col=['sea_level_pressure', 'wind_direction','wind_speed']
df_train.drop(unimportant_col,axis=1,inplace=True)

#time variable
df_train["timestamp"] = pd.to_datetime(df_train["timestamp"])
df_train["hour"] = df_train["timestamp"].dt.hour
df_train["weekday"] = df_train["timestamp"].dt.weekday
df_train["month"] = df_train["timestamp"].dt.month
df_train['year_built'] = df_train['year_built']-1900
df_train['square_feet'] = np.log1p(df_train['square_feet'])

dates_range = pd.date_range(start='2015-12-31', end='2019-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())

df_train['is_holiday'] = (df_train['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
del df_train["timestamp"]

df_train['meter_reading'] = np.log1p(df_train['meter_reading'])

# Memory reduce

In [11]:
#Based on this great kernel https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            print("min for this col: ",mn)
            print("max for this col: ",mx)
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

In [12]:
df_train ,_ = reduce_mem_usage(df_train)

Memory usage of properties dataframe is : 3104.011631011963  MB
******************************
Column:  building_id
dtype before:  int64
min for this col:  0
max for this col:  1448
dtype after:  uint16
******************************
******************************
Column:  meter
dtype before:  int64
min for this col:  0
max for this col:  3
dtype after:  uint8
******************************
******************************
Column:  meter_reading
dtype before:  float64
min for this col:  0.0
max for this col:  16.902211829285342
dtype after:  float32
******************************
******************************
Column:  site_id
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  primary_use
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  square_feet
dtype before:  float64
min for th

In [13]:
category = []
category = ['month','hour','meter','weekday','primary_use','site_id','building_id']
for col in category :
    df_train[col] = df_train[col].astype('category')



# Model

In [14]:
# force the model to use the weather data instead of dates, to avoid overfitting to the past history
features = [col for col in df_train.columns if col not in ['meter_reading', 'year', 'month', 'day', 'index','precip_depth_1_hr', 'floor_count','cloud_coverage',]]
target = 'meter_reading'
features

['building_id',
 'meter',
 'site_id',
 'primary_use',
 'square_feet',
 'year_built',
 'cnt_building_per_site',
 'cnt_building_per_site_prim',
 'sqr_mean_per_site',
 'sqr_mean_per_prim_site',
 'air_temperature',
 'dew_temperature',
 'hour',
 'weekday',
 'is_holiday']

In [ ]:
df_train = df_train.reset_index(drop=True)

folds = 3
seed = 42
kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
# oof_pred = np.zeros(train.shape[0])  # out of fold predictions
models = []
oof = np.zeros(len(df_train))
## stratify data by building_id
print('Light GBM Model')
for fold_, (trn_idx, val_idx)  in enumerate(kf.split(df_train, df_train['meter_reading'])):
        tr_x, tr_y = df_train[features].iloc[trn_idx], df_train[target][trn_idx]
        vl_x, vl_y = df_train[features].iloc[val_idx], df_train[target][val_idx]
        print({'train size':len(tr_x), 'eval size':len(vl_x)})
        print('fold : ',fold_)
        tr_data = lgb.Dataset(tr_x, label=tr_y)
        vl_data = lgb.Dataset(vl_x, label=vl_y) 
        reg= lgb.LGBMRegressor(n_estimators=6000,
                                learning_rate=0.05,
                                feature_fraction=0.7,
                                subsample=0.8,
                                num_leaves=40,
                                metric='rmse')
        reg.fit(tr_x, tr_y, eval_set=[(tr_x, tr_y),(vl_x, vl_y)],early_stopping_rounds=200,verbose=500)
        oof[val_idx] = reg.predict(df_train.iloc[val_idx][features])
        models.append(reg)
        gc.collect()
    
print('oof_RMSE : ' ,np.sqrt(mean_squared_error(oof, df_train['meter_reading'])))

Light GBM Model
{'train size': 13477400, 'eval size': 6738700}
fold :  0


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[500]	training's rmse: 1.02604	valid_1's rmse: 1.02772
[1000]	training's rmse: 0.99797	valid_1's rmse: 1.00051
[1500]	training's rmse: 0.976312	valid_1's rmse: 0.979712
[2000]	training's rmse: 0.959337	valid_1's rmse: 0.963654
[2500]	training's rmse: 0.945864	valid_1's rmse: 0.951121
[3000]	training's rmse: 0.934159	valid_1's rmse: 0.940238
[3500]	training's rmse: 0.922782	valid_1's rmse: 0.929747
[4000]	training's rmse: 0.914542	valid_1's rmse: 0.922296
[4500]	training's rmse: 0.906747	valid_1's rmse: 0.915414
[5000]	training's rmse: 0.899574	valid_1's rmse: 0.908989
[5500]	training's rmse: 0.891886	valid_1's rmse: 0.902082
[6000]	training's rmse: 0.886018	valid_1's rmse: 0.896971
{'train size': 13477400, 'eval size': 6738700}
fold :  1
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[500]	training's rmse: 1.01983	valid_1's rmse: 1.021
[1000]	training's rmse: 0.990778	valid_1's rmse: 0.993034
[1500]	training's

In [ ]:
print('oof_RMSE : ' ,np.sqrt(mean_squared_error(oof, df_train['meter_reading'])))

# predict

In [ ]:
df_test = pd.read_csv('/kaggle/input/ashrae-great-energy-predictor-iii-dataset/test.csv')
weather_test = pd.read_csv('/kaggle/input/ashrae-great-energy-predictor-iii-dataset/weather_test.csv')

print('df_test_shape :' , df_test.shape)
print('weather_test_shape :' , weather_test.shape)

In [ ]:
df_test = df_test.merge(building_metadata, on='building_id', how='left')
df_test = df_test.merge(weather_test, on=[ 'timestamp'], how='left')
# del weather_test
# del building_metadata
gc.collect()

print('df_test_merge shape: ',df_test.shape)

In [ ]:
df_test ,_ = reduce_mem_usage(df_test)

In [ ]:
#drop col
unimportant_col=['sea_level_pressure', 'wind_direction','wind_speed']
df_test.drop(unimportant_col,axis=1,inplace=True)

#time variable
df_test["timestamp"] = pd.to_datetime(df_test["timestamp"])
df_test["hour"] = df_test["timestamp"].dt.hour
df_test["weekday"] = df_test["timestamp"].dt.weekday
df_test["month"] = df_test["timestamp"].dt.month
df_test['year_built'] = df_test['year_built']-1900
df_test['square_feet'] = np.log1p(df_test['square_feet'])


df_test['is_holiday'] = (df_test['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)

del df_test["timestamp"]

In [ ]:
# category = ['month','hour','meter','weekend','primary_use','site_id','building_id']
for col in category :
    df_test[col] = df_test[col].astype('category')

In [ ]:
# split test data into batches
set_size = len(df_test)
iterations = 50
batch_size = set_size // iterations

print(set_size, iterations, batch_size)
assert set_size == iterations * batch_size

In [ ]:
meter_reading = []
for i in tqdm_notebook(range(iterations)):
    pos = i*batch_size
    fold_preds = [np.expm1(model.predict(df_test[features].iloc[pos : pos+batch_size])) for model in models]
    meter_reading.extend(np.mean(fold_preds, axis=0))

print(len(meter_reading))
assert len(meter_reading) == set_size

In [ ]:
sample_submission['meter_reading'] = np.clip(meter_reading, a_min=0, a_max=None) # clip min at zero
sample_submission.to_csv('submission_whatsyourcv3_0052_trncl.csv.gz',index=False,compression='gzip', float_format='%.4f', chunksize=25000)
sample_submission.head(15)

In [ ]:
sample_submission['meter_reading'] = np.clip(meter_reading, a_min=0, a_max=None) # clip min at zero
sample_submission.to_csv('submission_whatsyourcv3_0052_trncl.csv.gz',index=False,compression='gzip', float_format='%.4f', chunksize=25000)
sample_submission.head(15)

In [ ]:
!kaggle competitions submit -c ashrae-energy-prediction -f submission_whatsyourcv3_0052_trncl.csv.gz -m "lgbm old style"